# Assignment 2
# Ling Fei Zhang, 260985358
# Brandon Ma, 260983550

## Question 1

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from functions import SARSA
import time
import gymnasium as gym


Here, we will set a few parameters for our environment.

In [3]:
ALPHAS = [0.01, 0.1, 0.2]
GAMMA = 0.9
TEMPERATURE = [1, 10, 100]
EPISODES = 5500
SEEDS = np.arange(10)


Driver Code to test out the combination of each alpha with each temperature for SARSA.

In [6]:
env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=False)
env.reset()
sarsa = SARSA(env, 0.1, 1, GAMMA, EPISODES, False)
sarsa.simulate_episodes()
final_policy = sarsa.learned_policy
print("final policy:\n", final_policy.reshape(4,4)) #tabular format to better visualize
# print("total reward: ", rewards)

#uncomment this line if you want to vizualize the game
# env.visualize(final_policy, 10)

100%|██████████| 5500/5500 [00:01<00:00, 4467.81it/s]

final policy:
 [[2. 2. 1. 0.]
 [1. 1. 1. 2.]
 [2. 1. 1. 0.]
 [0. 2. 2. 0.]]


In [22]:
for alpha in ALPHAS:
    for temp in TEMPERATURE:
        env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=False)
        env.reset()
        sarsa = SARSA(env, alpha, temp, GAMMA, EPISODES, False)

### Effect of parameters on final training performance for SARSA

### Effect of parameters on final testing performance for SARSA

### Best parameter for SARSA

### Conclusion for SARSA

### Effect of parameters on final training performance for EXPECTED SARSA

### Effect of parameters on final testing performance for EXPECTED SARSA

### Best parameter for EXPECTED SARSA

### Conclusion for EXPECTED SARSA